In [ ]:
from datetime import datetime
import json
from notebookutils import mssparkutils


In [ ]:
%run Functions/ReferenceFunctions

In [ ]:
TenantID = "817ee1ce-6f1d-4c9d-adbe-d99f5c115c71"
SynapseSPNClientID = "5e73d77b-9b46-4bac-9ad3-2c93c1a82c74"
KeyVaultName = "https://czv-g-n-gendag-d-akv-02.vault.usgovcloudapi.net/"
ClientIDKeyVaultSecretName = "DAG-AzureSynapse-Virginia-ClientSecret"
AzureSQLJDBCURLSecretName = "DAG-AzureSQL-DAGTools-Virginia-JDBCURL-Secret"
# Step 1: Get the Azure SQL JDBC URL

tokenlibrary = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary  

AzureSQLjdbcurl = tokenlibrary.getSecret(f"{KeyVaultName}", f"{AzureSQLJDBCURLSecretName}", "ls_kv_dag")  


# Step 2: Get the Access Token

access_token,exec_statement,con = jdbc_authorization(KeyVaultName, AzureSQLjdbcurl, ClientIDKeyVaultSecretName, TenantID, SynapseSPNClientID, "SELECT 1") 

In [ ]:
def log_activity(pid, stage, status,start,end, size,filename):
    duration = (end - start).total_seconds()
    throughput = size/duration
    start = start.strftime("%Y%m%d %H:%M:%S")
    end = end.strftime("%Y%m%d %H:%M:%S")
    stmt = con.createStatement()
    insert = f"INSERT INTO dbo.pipeline_activity(id,stage,file_name,stat,start_time,end_time,duration,size,throughput,) VALUES({pid},{stage},{filename},{status},{start},{end},{duration},{size},{throughput})"
    stmt.executeUpdate(insert)
